In [6]:
import pickle
import pandas as pd

# Step 1: Load the classifier model from the existing pickle file
with open('model_ESMonly.pickle', 'rb') as model_file:
    classifier = pickle.load(model_file)

# Step 2: Load the CDR3 embeddings from the CSV file
# Assuming the CSV contains embeddings as rows/columns
cdr3_embeddings = pd.read_csv('cdr3_embeddings_esm3-small-2024-08.csv')

# Step 3: Combine the classifier and embeddings into a dictionary
combined_data = {
    'classifier': classifier,
    'cdr3_embeddings': cdr3_embeddings
}

# Step 4: Serialize (pickle) the combined data into a single pickle file
with open('model_with_embeddings.pkl', 'wb') as combined_file:
    pickle.dump(combined_data, combined_file)

print("Combined pickle file created successfully.")

/opt/conda/lib/python3.11/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator LogisticRegression from version 1.5.2 when using version 1.4.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


Combined pickle file created successfully.


In [1]:
import os
import pickle
import boto3
import sagemaker
from sagemaker.model import Model
from sagemaker.serverless import ServerlessInferenceConfig
import tarfile
import json
import requests
import botocore.exceptions

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


In [2]:
# Step 1: Prepare the Model Artifact
# ====================================

# Create a tar.gz file of the model directory
with tarfile.open("model_artifact.tar.gz", "w:gz") as tar:
    tar.add("tcr_model", arcname=".")

In [2]:
# Step 2: Upload Model Artifact to S3
# ====================================

# Initialize the boto3 S3 client
s3_client = boto3.client('s3')

# Define S3 bucket and path
bucket_name = 'tcr-prediction-us-east-1'  # Replace with your S3 bucket name
model_artifact_path = 'model_artifact.tar.gz'

# Create the S3 bucket if it doesn't exist
try:
    s3_client.head_bucket(Bucket=bucket_name)
    print(f"Bucket '{bucket_name}' already exists.")
except botocore.exceptions.ClientError as e:
    # If a 404 error is thrown, the bucket does not exist.
    if e.response['Error']['Code'] == '404':
        s3_client.create_bucket(
        Bucket=bucket_name
    )
        print(f"Bucket '{bucket_name}' created successfully.")
    else:
        raise e

# Upload the tar.gz file to S3
s3_client.upload_file(model_artifact_path, bucket_name, model_artifact_path)

print(f"Model artifact uploaded to s3://{bucket_name}/{model_artifact_path}")

Bucket 'tcr-prediction-us-east-1' already exists.
Model artifact uploaded to s3://tcr-prediction-us-east-1/model_artifact.tar.gz


In [3]:
# Step 3: Deploy the Model on SageMaker Serverless
# ================================================

# Initialize SageMaker session
sagemaker_session = sagemaker.Session()

# Define the SageMaker execution role (replace with your actual role ARN)
role = 'arn:aws:iam::340752820161:role/service-role/AmazonSageMaker-ExecutionRole-20241011T160996'  # Replace with your SageMaker role ARN

# Specify the S3 path of the model artifact
model_artifact = f's3://{bucket_name}/{model_artifact_path}'

# Use a pre-built SageMaker inference container image
# Replace with the correct AWS region where you want to deploy the model
region = 'us-east-1'
image_uri = '340752820161.dkr.ecr.us-east-1.amazonaws.com/tcr-epitope-predictor:v4v3'

# Create a SageMaker Model object
model = Model(
    model_data=model_artifact,
    role=role,
    image_uri=image_uri,
    entry_point="tcr_model/inference.py",  # The script name for inference
    sagemaker_session=sagemaker_session,
    env={
        "ESM3_TOKEN": ""  # Replace with your actual token
    }
)

# Specify the instance type and number of instances for real-time inference
instance_type = 'ml.m5.xlarge'  # Adjust based on your model's needs
initial_instance_count = 1  # The number of instances to launch

# Endpoint configuration name (same as before)
endpoint_name = 'tcr-epitope-predictor-endpoint'

# Configure serverless inference settings
# serverless_config = ServerlessInferenceConfig(
#     memory_size_in_mb=4096,  # Adjust this based on your model's needs
#     max_concurrency=10  # Adjust this to control concurrency
# )

# Endpoint Configuration Name
endpoint_name = 'tcr-epitope-predictor-endpoint'

# Delete the existing endpoint configuration if it exists
sm_client = boto3.client('sagemaker', region_name=region)
try:
    sm_client.describe_endpoint_config(EndpointConfigName=endpoint_name)
    print(f"Endpoint configuration '{endpoint_name}' exists. Deleting it now...")
    sm_client.delete_endpoint_config(EndpointConfigName=endpoint_name)
    print(f"Endpoint configuration '{endpoint_name}' deleted successfully.")
except sm_client.exceptions.ClientError as e:
    if e.response['Error']['Code'] == 'ValidationException':
        print(f"Endpoint configuration '{endpoint_name}' does not exist. Proceeding with deployment.")

# Deploy the model to a real-time endpoint
predictor = model.deploy(
    instance_type=instance_type,
    initial_instance_count=initial_instance_count,
    endpoint_name=endpoint_name
)

# # Deploy the serverless endpoint
# predictor = model.deploy(
#     serverless_inference_config=serverless_config,
#     endpoint_name=endpoint_name
# )

print(f"Endpoint '{endpoint_name}' deployed successfully.")

Endpoint configuration 'tcr-epitope-predictor-endpoint' exists. Deleting it now...
Endpoint configuration 'tcr-epitope-predictor-endpoint' deleted successfully.
--------------------------------------------------*

UnexpectedStatusException: Error hosting endpoint tcr-epitope-predictor-endpoint: Failed. Reason: The primary container for production variant AllTraffic did not pass the ping health check. Please check CloudWatch logs for this endpoint.. Try changing the instance type or reference the troubleshooting page https://docs.aws.amazon.com/sagemaker/latest/dg/async-inference-troubleshooting.html

In [ ]:
# Step 4: Test the Endpoint
# =========================

# Define the endpoint URL
endpoint_url = f"https://runtime.sagemaker.{region}.amazonaws.com/endpoints/{endpoint_name}/invocations"

# Define a sample payload for prediction
payload = {
    "epitopes": ["DIYKGMGPLLATVFKSV", "GMGPLLATVFKSV"]  # Replace with your test data
}

# Set the headers
headers = {
    "Content-Type": "application/json"
}

# Send a POST request to the deployed endpoint
response = requests.post(endpoint_url, headers=headers, data=json.dumps(payload))

# Print the response from the endpoint
if response.status_code == 200:
    print("Response:", response.json())
else:
    print("Error:", response.status_code, response.text)

In [2]:
import requests

region = 'us-east-1'
# Endpoint Configuration Name
endpoint_name = 'tcr-epitope-predictor-endpoint'

# Define the SageMaker endpoint URL for /ping
endpoint_url = f"https://runtime.sagemaker.{region}.amazonaws.com/endpoints/{endpoint_name}/ping"

# Send a GET request to the /ping endpoint
response = requests.get(endpoint_url)

# Print the response
if response.status_code == 200:
    print("Response:", response.json())
else:
    print("Error:", response.status_code, response.text)


Error: 404 <UnknownOperationException/>



In [3]:
import boto3
import json

# Initialize the SageMaker runtime client
sagemaker_runtime = boto3.client('sagemaker-runtime')

# Define the endpoint name
endpoint_name = 'tcr-epitope-predictor-endpoint'

# Define the input payload (epitopes)
payload = {
    "epitopes": ["DIYKGMGPLLATVFKSV", "GMGPLLATVFKSV"]
}

# Send the request to the endpoint
response = sagemaker_runtime.invoke_endpoint(
    EndpointName=endpoint_name,
    ContentType="application/json",
    Body=json.dumps(payload)
)

# Print the response from the model
result = json.loads(response['Body'].read().decode())
print("Response:", result)


ValidationError: An error occurred (ValidationError) when calling the InvokeEndpoint operation: Endpoint tcr-epitope-predictor-endpoint of account 340752820161 not found.

In [9]:
!pip install esm

In [12]:
from esm.sdk import client
from esm.sdk.api import ESMProtein, SamplingConfig

token = ""
model = client(
    model="esm3-small-2024-08",
    url="https://forge.evolutionaryscale.ai",
    token=token,
)

try:
    protein = ESMProtein(sequence="DIYKGMGPLLATVFKSV")
    protein_tensor = model.encode(protein)
    print("Encoding successful.")
except Exception as e:
    print(f"Error: {e}")


KeyboardInterrupt: 